<a href="https://colab.research.google.com/github/HiagoAD/GDI2/blob/master/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import calendar

## Load datasets


In [53]:
dengue = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2020.csv", encoding ="UTF-8")
emlurb = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2020.csv", encoding ="UTF-8")

dengue19 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2019.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb19 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2019.csv", encoding ="UTF-8")

dengue18 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2018.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb18 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2018.csv", encoding ="UTF-8")

dengue17 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2017.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb17 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2017.csv", encoding ="UTF-8")

dengue16 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2016.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb16 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2016.csv", encoding ="UTF-8")


b'Skipping line 108: expected 126 fields, saw 127\nSkipping line 1485: expected 126 fields, saw 127\nSkipping line 2584: expected 126 fields, saw 127\nSkipping line 4415: expected 126 fields, saw 127\nSkipping line 5659: expected 126 fields, saw 127\nSkipping line 6469: expected 126 fields, saw 127\n'
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (55,113) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Dados de 2020 - 2016


Iremos fazer os mesmos tratamentos anteriores para os dados de 2019 até 2016.

In [49]:
# Utilizamos apenas as colunas de bairro e de data do registro do caso
dengue_data = dengue[["no_bairro_residencia", "dt_notificacao"]]
dengue_data['dt_notificacao'] = pd.to_datetime(dengue_data['dt_notificacao'])

dengue_data19 = dengue19[["no_bairro_residencia", "dt_notificacao"]]
dengue_data19['dt_notificacao'] = pd.to_datetime(dengue_data19['dt_notificacao'])

dengue_data18 = dengue18[["no_bairro_residencia", "dt_notificacao"]]
dengue_data18['dt_notificacao'] = pd.to_datetime(dengue_data18['dt_notificacao'])

dengue_data17 = dengue17[["no_bairro_residencia", "dt_notificacao"]]
dengue_data17['dt_notificacao'] = pd.to_datetime(dengue_data17['dt_notificacao'])

dengue_data16 = dengue16[["no_bairro_residencia", "dt_notificacao"]]
dengue_data16['dt_notificacao'] = pd.to_datetime(dengue_data16['dt_notificacao'])

# Número de casos por mês

dengue_data = dengue_data.groupby([pd.Grouper(key="dt_notificacao", freq="M")]).size().reset_index(name="CASOS_DENGUE")
dengue_data = dengue_data.drop(dengue_data.index[[0]])

dengue_data19 = dengue_data19.groupby([pd.Grouper(key="dt_notificacao", freq="M")]).size().reset_index(name="CASOS_DENGUE")
dengue_data19 = dengue_data19.drop(dengue_data19.index[[12,13]])

dengue_data18 = dengue_data18.groupby([pd.Grouper(key="dt_notificacao", freq="M")]).size().reset_index(name="CASOS_DENGUE")
dengue_data18 = dengue_data18.drop(index=[12,13,14,15,16,17,18,19,20,21,22,23,24,25])

dengue_data17 = dengue_data17.groupby([pd.Grouper(key="dt_notificacao", freq="M")]).size().reset_index(name="CASOS_DENGUE")
dengue_data17 = dengue_data17.drop(index=[12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30])

dengue_data16 = dengue_data16.groupby([pd.Grouper(key="dt_notificacao", freq="M")]).size().reset_index(name="CASOS_DENGUE")
dengue_data16 = dengue_data16.drop(index=[12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [50]:
# Utilizamos apenas as colunas de bairro, data de demanda e tipo de serviço

emlurb_data = emlurb[["GRUPOSERVICO_DESCRICAO", "BAIRRO", "DATA_DEMANDA"]]
emlurb_data["DATA_DEMANDA"] = pd.to_datetime(emlurb_data["DATA_DEMANDA"])
emlurb_data = emlurb_data[(emlurb_data.GRUPOSERVICO_DESCRICAO == "DRENAGEM")]

emlurb_data19 = emlurb19[["GRUPOSERVICO_DESCRICAO", "BAIRRO", "DATA_DEMANDA", "SITUACAO"]]
emlurb_data19["DATA_DEMANDA"] = pd.to_datetime(emlurb_data19["DATA_DEMANDA"])
emlurb_data19 = emlurb_data19[(emlurb_data19.GRUPOSERVICO_DESCRICAO == "DRENAGEM")]

emlurb_data18 = emlurb18[["GRUPOSERVICO_DESCRICAO", "BAIRRO", "DATA_DEMANDA", "SITUACAO"]]
emlurb_data18["DATA_DEMANDA"] = pd.to_datetime(emlurb_data18["DATA_DEMANDA"])
emlurb_data18 = emlurb_data18[(emlurb_data18.GRUPOSERVICO_DESCRICAO == "DRENAGEM")]

emlurb_data17 = emlurb17[["GRUPOSERVICO_DESCRICAO", "BAIRRO", "DATA_DEMANDA", "SITUACAO"]]
emlurb_data17 = emlurb_data17[(emlurb_data17.GRUPOSERVICO_DESCRICAO == "DRENAGEM")]
emlurb_data17["DATA_DEMANDA"] = pd.to_datetime(emlurb_data17["DATA_DEMANDA"])

emlurb_data16 = emlurb16[["GRUPOSERVICO_DESCRICAO", "BAIRRO", "DATA_DEMANDA", "SITUACAO"]]
emlurb_data16["DATA_DEMANDA"] = pd.to_datetime(emlurb_data16["DATA_DEMANDA"])
emlurb_data16 = emlurb_data16[(emlurb_data16.GRUPOSERVICO_DESCRICAO == "DRENAGEM")]

# # Número de solicitações por mês
emlurb_data   = emlurb_data.groupby([pd.Grouper(key="DATA_DEMANDA", freq="M")]).size().reset_index(name="CHAMADOS_EMLURB")
emlurb_data19 = emlurb_data19.groupby([pd.Grouper(key="DATA_DEMANDA", freq="M")]).size().reset_index(name="CHAMADOS_EMLURB")
emlurb_data18 = emlurb_data18.groupby([pd.Grouper(key="DATA_DEMANDA", freq="M")]).size().reset_index(name="CHAMADOS_EMLURB")
emlurb_data17 = emlurb_data17.groupby([pd.Grouper(key="DATA_DEMANDA", freq="M")]).size().reset_index(name="CHAMADOS_EMLURB")
emlurb_data16 = emlurb_data16.groupby([pd.Grouper(key="DATA_DEMANDA", freq="M")]).size().reset_index(name="CHAMADOS_EMLURB")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [51]:
crossed_info = pd.merge(dengue_data, emlurb_data, left_on=["dt_notificacao"],right_on=["DATA_DEMANDA"])
crossed_info19 = pd.merge(dengue_data19, emlurb_data19, left_on=["dt_notificacao"],right_on=["DATA_DEMANDA"])
crossed_info18 = pd.merge(dengue_data18, emlurb_data18, left_on=["dt_notificacao"],right_on=["DATA_DEMANDA"])
crossed_info17 = pd.merge(dengue_data17, emlurb_data17, left_on=["dt_notificacao"],right_on=["DATA_DEMANDA"])
crossed_info16 = pd.merge(dengue_data16, emlurb_data16, left_on=["dt_notificacao"],right_on=["DATA_DEMANDA"])

#Crossed info coluna mes

crossed_info['mes']   = crossed_info.dt_notificacao.apply(lambda x: x.strftime('%m'))
crossed_info19['mes'] = crossed_info19.dt_notificacao.apply(lambda x: x.strftime('%m'))
crossed_info18['mes'] = crossed_info18.dt_notificacao.apply(lambda x: x.strftime('%m'))
crossed_info17['mes'] = crossed_info17.dt_notificacao.apply(lambda x: x.strftime('%m'))
crossed_info16['mes'] = crossed_info16.dt_notificacao.apply(lambda x: x.strftime('%m'))

# Análise para Casos de dengue x Meses de 2020 a 2016

crossed_info['mes'] = pd.to_numeric(crossed_info['mes'])
crossed_info19['mes'] = pd.to_numeric(crossed_info19['mes'])
crossed_info18['mes'] = pd.to_numeric(crossed_info18['mes'])
crossed_info17['mes'] = pd.to_numeric(crossed_info17['mes'])
crossed_info16['mes'] = pd.to_numeric(crossed_info16['mes'])

crossed_info['month'] = crossed_info['mes'].apply(lambda x: calendar.month_abbr[x])
crossed_info19['month'] = crossed_info19['mes'].apply(lambda x: calendar.month_abbr[x])
crossed_info18['month'] = crossed_info18['mes'].apply(lambda x: calendar.month_abbr[x])
crossed_info17['month'] = crossed_info17['mes'].apply(lambda x: calendar.month_abbr[x])
crossed_info16['month'] = crossed_info16['mes'].apply(lambda x: calendar.month_abbr[x])

In [52]:
#Merge info geral

data_temp_19 = crossed_info19
data_temp_19 = data_temp_19.drop(data_temp_19.index[[7,8,9,10,11]])

data_temp_18 = crossed_info18
data_temp_18 = data_temp_18.drop(data_temp_18.index[[7,8,9,10,11]])

data_temp_17 = crossed_info17
data_temp_17 = data_temp_17.drop(data_temp_17.index[[7,8,9,10,11]])

data_temp_16 = crossed_info16
data_temp_16 = data_temp_16.drop(data_temp_16.index[[7,8,9,10,11]])

data_temp_16 = data_temp_16[['CASOS_DENGUE', 'CHAMADOS_EMLURB', 'month']]
data_temp_17 = data_temp_17[['CASOS_DENGUE', 'CHAMADOS_EMLURB', 'month']]
data_temp_18 = data_temp_18[['CASOS_DENGUE', 'CHAMADOS_EMLURB', 'month']]
data_temp_19 = data_temp_19[['CASOS_DENGUE', 'CHAMADOS_EMLURB', 'month']]
data_temp_20 = crossed_info[['CASOS_DENGUE', 'CHAMADOS_EMLURB', 'month']]

merge_info = data_temp_20.append([data_temp_19,data_temp_18,data_temp_17,data_temp_16], ignore_index = True)



## Feature de predição

Texto.

## Separação de dados: Treino, Validação e Teste

Texto.

## Escolha de Algoritmos

*   Decision Tree
*   Random Forest
*   MLP
*   KNN



## MLFlow

Texto.

## Seleção de hiper-parâmetros

Texto.


## Diagnóstico de modelos

Texto.
